In [1]:
import math
import sys
from tqdm import tqdm
import torchaudio
import matplotlib.pyplot as plt
sys.path.append("/home/irene/Documents/audio-diffusion-pytorch-trainer")

from main.separation import ContextualSeparator, differential_with_gaussian, differential_with_dirac
from pathlib import Path
from audio_diffusion_pytorch import KarrasSchedule
import torch
from script.misc import hparams
import main.module_base
from IPython.display import Audio
device = torch.device("cuda")
ROOT_PATH = Path(".")
%load_ext autoreload
%autoreload 2

sampling_rate = 22050

# @markdown Generation length in seconds (will be rounded to be a power of 2 of sample_rate*length)
length = 10
length_samples = 2**math.ceil(math.log2(length * sampling_rate))

# @markdown Number of samples to generate
num_samples = 3

# @markdown Number of diffusion steps (higher tends to be better but takes longer to generate)
num_stems = 4

smin = 1e-4
smax = 1.0
rho = 7.0
sigma_schedule=KarrasSchedule(sigma_min=smin, sigma_max=smax, rho=rho)

def load_model(path):
  model = main.module_base.Model(**{**hparams, "in_channels": 4})
  model.load_state_dict(torch.load(path)["state_dict"])
  model.to(device);
  return model

def gamma_fn(x):
    return x / 0.5


In [2]:
model = load_model("/home/irene/Documents/audio-diffusion-pytorch-trainer/logs/ckpts/avid-darkness-164_epoch=419-valid_loss=0.015.ckpt")

In [3]:
def sisnr(preds: torch.Tensor, target: torch.Tensor, eps: float = 1e-5) -> torch.Tensor:
    alpha = (torch.sum(preds * target, dim=-1, keepdim=True) + eps) / (torch.sum(target**2, dim=-1, keepdim=True) + eps)
    target_scaled = alpha * target
    noise = target_scaled - preds
    s_target = torch.sum(target_scaled**2, dim=-1) + eps
    s_error = torch.sum(noise**2, dim=-1) + eps
    return 10 * torch.log10(s_target / s_error)

def sisnr_i(sep, ori, m):
    return sisnr(sep, ori) - sisnr(m, ori)

sources_names = ["bass", "drums", "guitar", "piano"]
generator1 = torch.Generator().manual_seed(1)
generator2 = torch.Generator().manual_seed(42)

In [4]:
# Tracks loading
track_list = []
srs = []
while len(track_list) < num_samples:
    track_n = torch.randint(low=1876, high=2098, size=(1,), dtype=torch.int, generator=generator2)[0]
    start_sample_n = torch.randint(high=120, size=(num_samples,), dtype=torch.int, generator=generator1)[0]
    start_sample = start_sample_n * sampling_rate
    stem_list = []
    is_silence_list = []
    mix = torch.zeros((1, 1, length_samples))
    flag = False
    for name in sources_names:
        try:
            s, sr = torchaudio.load(f'/home/irene/Documents/audio-diffusion-pytorch-trainer/data/Slack/test/{name}/Track0{track_n}.wav')
            s = torchaudio.functional.resample(s, orig_freq=sr, new_freq=sampling_rate)
            s = s.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
            assert(s.shape == mix.shape)
        except:
            flag = True
            break
        stem_list.append(s)
        is_silence_list.append((s == 0.).all()) # 1 means that is silence
        mix += s
    if flag or sum(is_silence_list) > 0 or mix.norm() < 10e-5:
        continue
    stems = torch.concat(stem_list, axis=1)
    track_list.append(stems)

tracks_gt = torch.concat(track_list, axis=0).to(device)
display(f"{tracks_gt.shape=}")

m = tracks_gt.sum(axis=1, keepdim=True)
display(m.shape)

'tracks_gt.shape=torch.Size([3, 4, 262144])'

torch.Size([3, 1, 262144])

In [ ]:
#display(Audio(tracks[0,0], rate = sampling_rate))
display(Audio(m[8].cpu().detach(), rate = sampling_rate))

In [6]:
#differential_fn = partial(differential_with_gaussian, gamma_fn=gamma_fn)
differential_fn = differential_with_dirac
separator = ContextualSeparator(model=model, 
                                stems=["bass", "drums", "guitar", "piano"], 
                                sigma_schedule=sigma_schedule,
                                num_resamples=3,
                                differential_fn=differential_fn,
                                s_churn=20.,
                                source_id=1)
num_steps = 10

In [ ]:
# GROUND TRUTH HINT (if fixed_sources_idx = [] is standard separation)

from main.separation import differential_with_gaussian
def generate_mask_and_sources(sources, fixed_sources_idx=[]):
    mobile_sources_idx = list(set([0,1,2,3]) - set(fixed_sources_idx))
    mobile_sources_idx.sort()
    inpaint                = torch.randn(num_samples, num_stems, length_samples).to(device)
    if len(fixed_sources_idx) > 0:
        inpaint[:, fixed_sources_idx, :]       = sources[:, fixed_sources_idx, :]
    inpaint_mask           = torch.ones_like(inpaint)
    inpaint_mask[:, mobile_sources_idx, :] = 0.
    return inpaint, inpaint_mask

sources_idx=torch.arange(4, device=device)

fixed_sources_idx = []
inpaint, inpaint_mask = generate_mask_and_sources(sources=tracks_gt, fixed_sources_idx=fixed_sources_idx)
separations_hint = separator.separate_with_hint(
    mixture=m,
    source_with_hint=inpaint,
    mask=inpaint_mask,
    num_steps=num_steps,
)
sources = torch.cat([separations_hint["bass"], separations_hint["drums"],
                     separations_hint["guitar"], separations_hint["piano"]], axis=1).to(device)

In [ ]:
# Plot results
sources_names = ["bass", "drums", "guitar", "piano"]
ss = []
num_chunks = 12
length_chunks = length_samples // num_chunks
for i in range(4):
    chunk_mean = 0
    chunk_std = 0
    for j in range(num_chunks):
        start = j*length_chunks
        end = (j+1)*length_chunks
        s = sisnr_i(separations_hint[sources_names[i]][:,:,start:end], 
                    tracks_gt[:,[i],start:end], m[:,:,start:end].to(device))
        chunk_mean += s.mean()
        chunk_std += s.std()
    print(f"{sources_names[i]}: mean = {chunk_mean/num_chunks}, std = {chunk_std/num_chunks}\n")
    ss.append(s)
ss = torch.stack(ss)
mobile_sources_idx = list(set([0, 1, 2, 3]) - set(fixed_sources_idx))
mean = sum(ss[mobile_sources_idx]) / len(mobile_sources_idx)
print(f"overall (exclude fix sources) mean = {mean.mean().tolist()}, std = {mean.std().tolist()}")

In [ ]:
T = 16
p_max = (T - 1) / T
p_min = p_max * (1 - 0.95)
def schedule_prob(t, T, alpha=0.95, p_min=p_min, p_max=p_max):
    p_t = 1 - max(0, p_max - (t - 1) * (p_max - p_min) / (alpha * (T - 1)))
    return p_t

plt.plot([schedule_prob(i, T) for i in range(T)])

In [ ]:
# GIBSS
def generate_mask_and_sources(sources, fixed_sources_idx=[]):
    mobile_sources_idx = list(set([0, 1, 2, 3]) - set(fixed_sources_idx))
    mobile_sources_idx.sort()
    inpaint                = torch.randn(num_samples, num_stems, length_samples).to(device)
    inpaint[:, fixed_sources_idx, :]       = sources[:, fixed_sources_idx, :]
    inpaint_mask           = torch.ones_like(inpaint)
    inpaint_mask[:, mobile_sources_idx, :] = 0.
    return inpaint, inpaint_mask

sources_idx=torch.arange(4, device=device)

for i in tqdm(range(T)):
    p = schedule_prob(i, T)
    mask = torch.bernoulli(torch.ones(4, device=device) * p).bool()
    fixed_sources_idx = sources_idx[mask]
    inpaint, inpaint_mask = generate_mask_and_sources(sources=tracks_gt, fixed_sources_idx=fixed_sources_idx)
    separations_hint = separator.separate_with_hint(
        mixture=m,
        num_steps=num_steps,
        mask=inpaint_mask,
        source_with_hint=inpaint,
    )
    sources = torch.cat([separations_hint["bass"], separations_hint["drums"],
                     separations_hint["guitar"], separations_hint["piano"]], axis=1).to(device)
fixed_sources_idx = [] # For computing metrics
# separations_hint_list.append(separations_hint)

In [ ]:
display(Audio(separations_hint["drums"][1].squeeze().squeeze().cpu().detach(), rate = sampling_rate))
display(Audio(tracks_gt[1][1].squeeze().squeeze().cpu().detach(), rate = sampling_rate))

Resample
1) 1, (3, 10) resample I ordine 100, (300, 1000) step
2) a) 1 resample II ordine 10, 100 step b) provare nuovi sampler shneider
3) dirac con variazioni sorgente
4) dirac 4 sep con ogni volta una sorgente diversa fissata
5) calcolare la media dei gradienti
6) likelihood

# Likelihood test

In [5]:
from main.separation import log_likelihood_song, log_likelihood_crawford

In [8]:
f = model.model.diffusion.denoise_fn
p_gt = log_likelihood_crawford(f, tracks_gt[[2]], smin, 1)
#p_rand = log_likelihood(f, torch.randn_like(tracks_gt[[0]]), 0.002, 1)[0].exp()

x_min=tensor([[[ 0.0726,  0.0698,  0.0672,  ..., -0.0291, -0.0262, -0.0225],
         [ 0.0191,  0.0185,  0.0173,  ...,  0.0005,  0.0006,  0.0010],
         [ 0.0360,  0.0309,  0.0254,  ..., -0.0142, -0.0107, -0.0074],
         [-0.0666, -0.0601, -0.0563,  ..., -0.0336, -0.0275, -0.0229]]],
       device='cuda:0')
t=tensor([1.0000e-04, 1.0000e+00], device='cuda:0')
----------------------------------------------------
tensor([3.7598e+08], device='cuda:0')


TypeError: unsupported operand type(s) for /: 'tuple' and 'Tensor'

In [36]:
mm = tracks_gt[[1]] + 0.01 * torch.randn_like(tracks_gt[[1]]).sum(1, keepdim=True)
display(Audio(mm.squeeze(0).cpu().detach(), rate = sampling_rate))

In [16]:
sol = p_gt[2]

In [17]:
sol[1]

tensor([[      0.],
        [6892697.]], device='cuda:0')